https://github.com/aymericdamien/TensorFlow-Examples/blob/master/examples/3_NeuralNetworks/gan.py
https://stackoverflow.com/questions/40994583/how-to-implement-tensorflows-next-batch-for-own-data

In [1]:
import xml.etree.ElementTree as ET
import csv


def get_labels_dict(data_path):
    labels_dict = {}
    with open(data_path + 'sci_labels.csv', 'r') as f:
        file = csv.reader(f)
        for row in file:
            labels_dict[row[0]] = row[1]
    return labels_dict


def get_features_labels(root, labels_dict):
    corpus = [] # each row is a string formed from all messages in a conversations
    labels = [] # each row is 0 or 1, corresponds to label for same row in corpus

    for conversation in root:
        string = " "
        for message in conversation:
            text = message.find('text').text
            if text is not None:
                string = string + "\r\n" + text 
        corpus.append(string)
        labels.append(int(labels_dict[conversation.get('id')]))
    return corpus, labels

In [2]:
train_data_path = '../../data/svm_training_data/'
training_xml = ET.parse(train_data_path + 'training_data.xml')
train_root = training_xml.getroot()

test_data_path = '../../data/svm_test_data/'
test_data_src = '../../data/pan12-sexual-predator-identification-test-corpus-2012-05-21/'
test_xml = ET.parse(test_data_src + 'pan12-sexual-predator-identification-test-corpus-2012-05-17.xml')
test_root = test_xml.getroot()

train_corpus, train_labels = get_features_labels(train_root, get_labels_dict(train_data_path))
test_corpus, test_labels = get_features_labels(test_root, get_labels_dict(test_data_path))

train_corpus_norm = []
train_corpus_susp = []
train_labels_norm = []
train_labels_susp = []
for index in range(len(train_corpus)):
    if train_labels[index] == 1:
        train_corpus_susp.append(train_corpus[index])
        train_labels_susp.append(train_labels[index])
    else:
        train_corpus_norm.append(train_corpus[index])
        train_labels_norm.append(train_labels[index])

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
from sklearn.preprocessing import MaxAbsScaler
import scipy
# from sklearn.model_selection import train_test_split
import numpy as np

print('TODO: PLEASE REMOVE LIMITATION OF FIRST 1000 TEST CASES DONE DUE TO MEMORY ISSUES')

vectorizer = TfidfVectorizer()
_ = vectorizer.fit_transform(train_corpus)
X_train_norm = vectorizer.transform(train_corpus_norm)
X_train_susp = vectorizer.transform(train_corpus_susp)
# limiting to 1000 test cases due to memory issues
X_test = vectorizer.transform(test_corpus[:1000])

X_train_norm = scipy.sparse.csr_matrix(X_train_norm, dtype=np.float32)
y_train_norm = np.array(train_labels_norm)
X_train_susp = scipy.sparse.csr_matrix(X_train_susp, dtype=np.float32)
y_train_susp = np.array(train_labels_susp)
X_test = scipy.sparse.csr_matrix(X_test, dtype=np.float32)
y_test = np.array(test_labels[:1000])

print(np.min(X_train_norm[:][0]))
print(np.max(X_train_norm[:][0]))

scaler = MaxAbsScaler()
X_train_norm = scaler.fit_transform(X_train_norm)
X_train_susp = scaler.fit_transform(X_train_susp)
X_test = scaler.fit_transform(X_test)
X_train_norm.data -= 0.5
X_train_susp.data -= 0.5
X_test.data -= 0.5
X_train_norm.data *= 2
X_train_susp.data *= 2
X_test.data *= 2

print(np.min(X_train_norm[:][0]))
print(np.max(X_train_norm[:][0]))

# X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.3, random_state=87)
# print("Train data shape:{}\r\nTest data shape:{}".format(X_train.shape, X_test.shape))

TODO: PLEASE REMOVE LIMITATION OF FIRST 1000 TEST CASES DONE DUE TO MEMORY ISSUES
0.0
0.44709805
-0.96498847
1.0


In [4]:
# print(X_train_norm[:][0])
print(np.mean(X_train_norm[:][0]))

-0.0010184544


In [5]:
""" Generative Adversarial Networks (GAN).
Using generative adversarial networks (GAN) to generate digit images from a
noise distribution.
References:
    - Generative adversarial nets. I Goodfellow, J Pouget-Abadie, M Mirza,
    B Xu, D Warde-Farley, S Ozair, Y. Bengio. Advances in neural information
    processing systems, 2672-2680.
    - Understanding the difficulty of training deep feedforward neural networks.
    X Glorot, Y Bengio. Aistats 9, 249-256
Links:
    - [GAN Paper](https://arxiv.org/pdf/1406.2661.pdf).
    - [MNIST Dataset](http://yann.lecun.com/exdb/mnist/).
    - [Xavier Glorot Init](www.cs.cmu.edu/~bhiksha/courses/deeplearning/Fall.../AISTATS2010_Glorot.pdf).
Author: Aymeric Damien
Project: https://github.com/aymericdamien/TensorFlow-Examples/
"""

from __future__ import division, print_function, absolute_import

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

# Import MNIST data
# from tensorflow.examples.tutorials.mnist import input_data
# mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

# Training Params
num_steps = 1000
batch_size = 100 #128
learning_rate = 0.0002

# Network Params
image_dim = X_train_norm.shape[1] #784 # 28*28 pixels
gen_hidden_dim = 10 #256
gen_hidden_dim2 = 10
disc_hidden_dim = 10 #256
noise_dim = X_train_norm.shape[1] # 100 # Noise data points
print(X_train_norm.shape[1])

# A custom initialization (see Xavier Glorot init)
def glorot_init(shape):
    return tf.random_normal(shape=shape, stddev=1. / tf.sqrt(shape[0] / 2.))

# Store layers weight & bias
weights = {
    'gen_hidden1': tf.Variable(glorot_init([noise_dim, gen_hidden_dim])),
    'gen_hidden2': tf.Variable(glorot_init([gen_hidden_dim, gen_hidden_dim2])),
    'gen_out': tf.Variable(glorot_init([gen_hidden_dim, image_dim])),
    'disc1_hidden1': tf.Variable(glorot_init([image_dim, disc_hidden_dim])),
    'disc1_out': tf.Variable(glorot_init([disc_hidden_dim, 1])),
    'disc2_hidden1': tf.Variable(glorot_init([image_dim, disc_hidden_dim])),
    'disc2_out': tf.Variable(glorot_init([disc_hidden_dim, 1])),
}
biases = {
    'gen_hidden1': tf.Variable(tf.zeros([gen_hidden_dim])),
    'gen_hidden2': tf.Variable(tf.zeros([gen_hidden_dim2])),
    'gen_out': tf.Variable(tf.zeros([image_dim])),
    'disc1_hidden1': tf.Variable(tf.zeros([disc_hidden_dim])),
    'disc1_out': tf.Variable(tf.zeros([1])),
    'disc2_hidden1': tf.Variable(tf.zeros([disc_hidden_dim])),
    'disc2_out': tf.Variable(tf.zeros([1])),
}


# Generator
def generator(x):
    hidden_layer = tf.matmul(x, weights['gen_hidden1'])
    hidden_layer = tf.add(hidden_layer, biases['gen_hidden1'])
    hidden_layer = tf.nn.leaky_relu(hidden_layer)
    hidden_layer2 = tf.matmul(hidden_layer, weights['gen_hidden2'])
    hidden_layer2 = tf.add(hidden_layer2, biases['gen_hidden2'])
    hidden_layer2 = tf.nn.leaky_relu(hidden_layer2)
    out_layer = tf.matmul(hidden_layer2, weights['gen_out'])
#     out_layer = tf.matmul(hidden_layer, weights['gen_out'])
    out_layer = tf.add(out_layer, biases['gen_out'])
    out_layer = tf.nn.tanh(out_layer)
    return out_layer


# Discriminator
def discriminator_SCI(x, reuse=False): # is D in paper
#     https://mlnotebook.github.io/post/GAN4/
    with tf.variable_scope("disc_SCI") as scope:
        if reuse:
            scope.reuse_variables()
        hidden_layer = tf.matmul(x, weights['disc1_hidden1'])
        hidden_layer = tf.add(hidden_layer, biases['disc1_hidden1'])
        hidden_layer = tf.nn.relu(hidden_layer)
        out_layer = tf.matmul(hidden_layer, weights['disc1_out'])
        out_layer = tf.add(out_layer, biases['disc1_out'])
        out_layer = tf.nn.sigmoid(out_layer)
        return out_layer

def discriminator_gvr(x, reuse=False): # is D prime in paper, discriminator_generated_vs_real
    with tf.variable_scope("disc_gvr") as scope:
        if reuse:
            scope.reuse_variables()
        hidden_layer = tf.matmul(x, weights['disc2_hidden1'])
        hidden_layer = tf.add(hidden_layer, biases['disc2_hidden1'])
        hidden_layer = tf.nn.relu(hidden_layer)
        out_layer = tf.matmul(hidden_layer, weights['disc2_out'])
        out_layer = tf.add(out_layer, biases['disc2_out'])
        out_layer = tf.nn.sigmoid(out_layer)
        return out_layer

def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , data.shape[0])
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i].toarray() for i in idx]
    labels_shuffle = [labels[ i] for i in idx]
    data_shuffle = scipy.sparse.csr_matrix(data_shuffle)
    print(data_shuffle.shape)
    return data_shuffle, labels_shuffle
#     print(len(data_shuffle[0]))
#     return scipy.sparse.csr_matrix(data_shuffle), np.asarray(labels_shuffle)
#     return np.asarray(data_shuffle), np.asarray(labels_shuffle)

# https://stackoverflow.com/questions/40896157/scipy-sparse-csr-matrix-to-tensorflow-sparsetensor-mini-batch-gradient-descent
def convert_sparse_matrix_to_sparse_tensor(X):
    coo = X.tocoo()
    indices = np.mat([coo.row, coo.col]).transpose()
    return tf.SparseTensor(indices, coo.data, coo.shape)

121394
Instructions for updating:
Colocations handled automatically by placer.


In [6]:
# Build Networks
# Network Inputs
gen_input = tf.placeholder(tf.float32, shape=[None, noise_dim], name='input_noise')
# disc_input_normal = tf.placeholder(tf.float32, shape=[None, image_dim], name='disc_input_normal')
# disc_input_real_susp = tf.placeholder(tf.float32, shape=[None, image_dim], name='disc_input_real_susp')
# # disc_input_fake_susp = tf.placeholder(tf.float32, shape=[None, image_dim], name='disc_input_fake_susp')

# tf.float32
# https://towardsdatascience.com/how-to-use-dataset-in-tensorflow-c758ef9e4428
# x_norm = tf.sparse_placeholder(tf.float64)
# x_susp = tf.sparse_placeholder(tf.float64)
x_norm, x_susp = tf.placeholder(tf.float32, shape=[None, image_dim]), tf.placeholder(tf.float32, shape=[None, image_dim])
features_noise = tf.placeholder(tf.float32, shape=[None, image_dim])
dataset_norm = tf.data.Dataset.from_tensor_slices(x_norm).repeat().batch(batch_size)
dataset_susp = tf.data.Dataset.from_tensor_slices(x_susp).repeat().batch(batch_size)
# train_data = (np.random.sample((100,2)), np.random.sample((100,1)))
# test_data = (np.array([[1,2]]), np.array([[0]]))
iter_norm = dataset_norm.make_initializable_iterator()
iter_susp = dataset_susp.make_initializable_iterator()
features_norm = iter_norm.get_next()
features_susp = iter_susp.get_next()

# Build Generator Network
# gen_sample_pre = generator(features_susp)
# gen_sample_pre = generator(gen_input)
gen_sample = generator(gen_input)

# Build 2 Discriminator Networks (one from noise input, one from generated samples)
# disc_SCI_normal = discriminator_SCI(disc_input_normal)
# disc_SCI_susp_real = discriminator_SCI(disc_input_real_susp)
# disc_SCI_susp_fake = discriminator_SCI(gen_sample)
# disc_gvr_real = discriminator_gvr(disc_input_real_susp)
# disc_gvr_fake = discriminator_gvr(gen_sample)

disc_SCI_normal = discriminator_SCI(features_norm)
disc_SCI_susp_real = discriminator_SCI(features_susp)# + features_noise)
disc_SCI_susp_fake = discriminator_SCI(gen_sample, reuse=True)
disc_gvr_real = discriminator_gvr(features_susp + features_noise)
disc_gvr_fake = discriminator_gvr(gen_sample, reuse=True)


# Build Loss
gen_loss_pre = tf.losses.mean_squared_error(features_susp, gen_sample)
# gen_loss = -tf.reduce_mean(tf.log(disc_SCI_susp_fake + 1e-8) + tf.log(disc_gvr_fake + 1e-8))
gen_loss = -tf.reduce_mean(tf.log(disc_gvr_fake + 1e-8))
disc_SCI_loss = -tf.reduce_mean(tf.log(disc_SCI_normal + 1e-8) + tf.log(1. - disc_SCI_susp_real + 1e-8) + tf.log(1. - disc_SCI_susp_fake + 1e-8))
disc_SCI_loss_pre = -tf.reduce_mean(tf.log(disc_SCI_normal + 1e-8) + tf.log(1. - disc_SCI_susp_real + 1e-8))
disc_gvr_loss = -tf.reduce_mean(tf.log(disc_gvr_real + 1e-8) + tf.log(1. - disc_gvr_fake + 1e-8))

# https://blog.paperspace.com/implementing-gans-in-tensorflow/
# disc_gvr_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=disc_gvr_real,labels=tf.ones_like(disc_gvr_real)) +\
#                            tf.nn.sigmoid_cross_entropy_with_logits(logits=disc_gvr_fake,labels=tf.zeros_like(disc_gvr_fake)))
# gen_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=disc_gvr_fake,labels=tf.ones_like(disc_gvr_fake)))
# disc_SCI_loss_pre = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=disc_SCI_normal,labels=tf.ones_like(disc_SCI_normal)) +\
#                            tf.nn.sigmoid_cross_entropy_with_logits(logits=disc_SCI_susp_real,labels=tf.zeros_like(disc_SCI_susp_real)))
# disc_SCI_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=disc_SCI_normal,labels=tf.ones_like(disc_SCI_normal)) +\
#                            tf.nn.sigmoid_cross_entropy_with_logits(logits=disc_SCI_susp_real,labels=tf.zeros_like(disc_SCI_susp_real)) + \
#                               tf.nn.sigmoid_cross_entropy_with_logits(logits=disc_SCI_susp_fake,labels=tf.zeros_like(disc_SCI_susp_fake)))
# experimenting with flipping the labels
# disc_gvr_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=disc_gvr_real,labels=tf.zeros_like(disc_gvr_real)) +\
#                            tf.nn.sigmoid_cross_entropy_with_logits(logits=disc_gvr_fake,labels=tf.ones_like(disc_gvr_fake)))
# gen_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=disc_gvr_fake,labels=tf.zeros_like(disc_gvr_fake)))
# disc_SCI_loss_pre = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=disc_SCI_normal,labels=tf.zeros_like(disc_SCI_normal)) +\
#                            tf.nn.sigmoid_cross_entropy_with_logits(logits=disc_SCI_susp_real,labels=tf.ones_like(disc_SCI_susp_real)))
# disc_SCI_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=disc_SCI_normal,labels=tf.zeros_like(disc_SCI_normal)) +\
#                            tf.nn.sigmoid_cross_entropy_with_logits(logits=disc_SCI_susp_real,labels=tf.ones_like(disc_SCI_susp_real)) + \
#                               tf.nn.sigmoid_cross_entropy_with_logits(logits=disc_SCI_susp_fake,labels=tf.ones_like(disc_SCI_susp_fake)))


# Build Optimizers
# optimizer_gen_pre = tf.train.AdamOptimizer(learning_rate=learning_rate)
optimizer_gen = tf.train.AdamOptimizer(learning_rate=learning_rate*10)
# optimizer_disc_SCI = tf.train.AdamOptimizer(learning_rate=learning_rate)
# optimizer_disc_gvr = tf.train.AdamOptimizer(learning_rate=learning_rate)
optimizer_disc_SCI = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
optimizer_disc_gvr = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)

# Training Variables for each optimizer
# By default in TensorFlow, all variables are updated by each optimizer, so we
# need to precise for each one of them the specific variables to update.
# Generator Network Variables
gen_vars = [weights['gen_hidden1'], weights['gen_out'],
            biases['gen_hidden1'], biases['gen_out']]
# Discriminator Network Variables
disc_SCI_vars = [weights['disc1_hidden1'], weights['disc1_out'],
            biases['disc1_hidden1'], biases['disc1_out']]
disc_gvr_vars = [weights['disc2_hidden1'], weights['disc2_out'],
            biases['disc2_hidden1'], biases['disc2_out']]

# Create training operations
train_gen_pre = optimizer_gen.minimize(gen_loss_pre, var_list=gen_vars)
train_gen = optimizer_gen.minimize(gen_loss, var_list=gen_vars)
train_disc_SCI = optimizer_disc_SCI.minimize(disc_SCI_loss, var_list=disc_SCI_vars)
train_disc_SCI_pre = optimizer_disc_SCI.minimize(disc_SCI_loss_pre, var_list=disc_SCI_vars)
train_disc_gvr = optimizer_disc_gvr.minimize(disc_gvr_loss, var_list=disc_gvr_vars)

# Add ops to save and restore all the variables.
saver = tf.train.Saver() #tf.train.Saver(max_to_keep=1)

print(X_train_norm.shape)
print(X_train_susp.shape)

Instructions for updating:
Use tf.cast instead.
(13802, 121394)
(901, 121394)


In [ ]:
import datetime

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

# options to restore saved model
restore = False
filename = ''

# Start training
with tf.Session() as sess:

    if restore == True and len(filename) > 0:
        # Restore variables from disk.
        saver.restore(sess, filename)
        print("Model restored.")
    else:
        # Run the initializer
        sess.run(init)
        print('Initializing dataset pipeline variables')
        sess.run([iter_norm.initializer, iter_susp.initializer], feed_dict={ x_norm: X_train_norm.todense(), x_susp: X_train_susp.todense()})
    
#     print('Pretraining of generator starting.')
#     for i in range(1, num_steps*100+1):
#         z = np.random.uniform(-1., 1., size=[batch_size, noise_dim])
#         feed_dict = {gen_input: z}
#         _, glpre = sess.run([train_gen_pre, gen_loss_pre], feed_dict=feed_dict)
#         print('Step %i: Generator Loss: %f' % (i, glpre))
        
    print('Pretraining of disc_SCI starting.')
    for i in range(1, num_steps+1):
        _, dscil = sess.run([train_disc_SCI_pre, disc_SCI_loss_pre])
        print('Step %i: Disc SCI Loss: %f' % (i, dscil))
        if i % 100 == 0:
            # Save the variables to disk.
            filename = '../../models/GAN_scionly_' + "{:.2f}loss_{}steps_".format(dscil, i) + datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S") + '.ckpt' 
            save_path = saver.save(sess, filename)
            print("Model saved in path: %s" % save_path)
            
    print('Pretraining of generator and disc_gvr starting.')
    for i in range(1, num_steps*100+1):
        z = np.random.uniform(-1., 1., size=[batch_size, noise_dim])
        feat_noise = np.random.uniform(-0.01, 0.01, size=[batch_size, noise_dim])
        feed_dict = {gen_input: z, features_noise: feat_noise}
        _, _, gl, dgvrl = sess.run([train_gen, train_disc_gvr, gen_loss, disc_gvr_loss], feed_dict=feed_dict)
        _, gl = sess.run([train_gen, gen_loss], feed_dict=feed_dict)
        print('Step %i: Generator Loss: %f, Disc gvr Loss: %f' % (i, gl, dgvrl))
        if i % 100 == 0:
            # Save the variables to disk.
            filename = '../../models/GAN_gengvr_' + "{:.2f}loss_{}steps_".format(gl, i) + datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")  + '.ckpt'
            save_path = saver.save(sess, filename)
            print("Model saved in path: %s" % save_path)
    
    print('Training of generator and both disc_gvr, disc_sci starting.')
    for i in range(1):#, num_steps*100+1):
        z = np.random.uniform(-1., 1., size=[batch_size, noise_dim])
        feat_noise = np.random.uniform(-0.01, 0.01, size=[batch_size, noise_dim])
        feed_dict = {gen_input: z, features_noise: feat_noise}
        _, _, _, gl, dgvrl, dscil = sess.run([train_gen, train_disc_gvr, train_disc_SCI, \
                                    gen_loss, disc_gvr_loss, disc_SCI_loss], feed_dict=feed_dict)
#         _, gl = sess.run([train_gen, gen_loss], feed_dict=feed_dict)
        print('Step %i: Generator Loss: %f, Disc gvr Loss: %f, Disc sci Loss: %f' % (i, gl, dgvrl, dscil))
        if i % 100 == 0:
            # Save the variables to disk.
            filename = './models/GAN/wholeGAN/GAN_whole_' + \
            "{:.2f}gl_{:.2f}dgvrl_{:.2f}dscil_{}steps_".format(gl, dgvrl, dscil, i) + datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")  + '.ckpt'
            save_path = saver.save(sess, filename)
            print("Model saved in path: %s" % save_path)
    
#     print('Starting actual training')
#     for i in range(1, num_steps+1):
#         # Generate noise to feed to the generator
#         z = np.random.uniform(-1., 1., size=[batch_size, noise_dim])
#         # Train
# #         feed_dict = {disc_input: batch_x, gen_input: z}
#         feed_dict = {gen_input: z}
#         _, _, _, gl, dSCIl, dgvrl = sess.run([train_gen, train_disc_SCI, train_disc_gvr, gen_loss, disc_SCI_loss, disc_gvr_loss],
#                                 feed_dict=feed_dict)
# #         if i % 1000 == 0 or i == 1:
#         print('Step %i: Generator Loss: %f, Disc SCI Loss: %f, Disc gvr Loss: %f' % (i, gl, dSCIl, dgvrl))

#     print("Finished Training")
#     # test SCI
#     sess.run(iter_norm.initializer, feed_dict={ x_norm: X_test.todense()})
#     y_pred = sess.run(disc_SCI_normal)
#     print(metrics.accuracy_score(y_test, y_pred))
#     # Generate images from noise, using the generator network.
#     f, a = plt.subplots(4, 10, figsize=(10, 4))
#     for i in range(10):
#         # Noise input.
#         z = np.random.uniform(-1., 1., size=[4, noise_dim])
#         g = sess.run([gen_sample], feed_dict={gen_input: z})
#         g = np.reshape(g, newshape=(4, 28, 28, 1))
#         # Reverse colours for better display
#         g = -1 * (g - 1)
#         for j in range(4):
#             # Generate image from noise. Extend to 3 channels for matplot figure.
#             img = np.reshape(np.repeat(g[j][:, :, np.newaxis], 3, axis=2),
#                              newshape=(28, 28, 3))
#             a[j][i].imshow(img)

#     f.show()
#     plt.draw()
#     plt.waitforbuttonpress()

Initializing dataset pipeline variables
Pretraining of disc_SCI starting.
Step 1: Disc SCI Loss: 1.385687
Step 2: Disc SCI Loss: 1.388105
Step 3: Disc SCI Loss: 1.387123
Step 4: Disc SCI Loss: 1.384549
Step 5: Disc SCI Loss: 1.386093
Step 6: Disc SCI Loss: 1.383520
Step 7: Disc SCI Loss: 1.390334
Step 8: Disc SCI Loss: 1.388534
Step 9: Disc SCI Loss: 1.384172
Step 10: Disc SCI Loss: 1.385192
Step 11: Disc SCI Loss: 1.388722
Step 12: Disc SCI Loss: 1.385078
Step 13: Disc SCI Loss: 1.384291
Step 14: Disc SCI Loss: 1.387928
Step 15: Disc SCI Loss: 1.385878
Step 16: Disc SCI Loss: 1.390057
Step 17: Disc SCI Loss: 1.383392
Step 18: Disc SCI Loss: 1.384451
Step 19: Disc SCI Loss: 1.385403
Step 20: Disc SCI Loss: 1.383814
Step 21: Disc SCI Loss: 1.383994
Step 22: Disc SCI Loss: 1.383268
Step 23: Disc SCI Loss: 1.384500
Step 24: Disc SCI Loss: 1.383506
Step 25: Disc SCI Loss: 1.386769
Step 26: Disc SCI Loss: 1.383582
Step 27: Disc SCI Loss: 1.382347
Step 28: Disc SCI Loss: 1.383007
Step 29: Di

Step 238: Disc SCI Loss: 1.349314
Step 239: Disc SCI Loss: 1.343884
Step 240: Disc SCI Loss: 1.344044
Step 241: Disc SCI Loss: 1.351105
Step 242: Disc SCI Loss: 1.343028
Step 243: Disc SCI Loss: 1.343262
Step 244: Disc SCI Loss: 1.344739
Step 245: Disc SCI Loss: 1.345821
Step 246: Disc SCI Loss: 1.346942
Step 247: Disc SCI Loss: 1.339335
Step 248: Disc SCI Loss: 1.344068
Step 249: Disc SCI Loss: 1.344507
Step 250: Disc SCI Loss: 1.345416
Step 251: Disc SCI Loss: 1.338242
Step 252: Disc SCI Loss: 1.344411
Step 253: Disc SCI Loss: 1.344210
Step 254: Disc SCI Loss: 1.345347
Step 255: Disc SCI Loss: 1.344889
Step 256: Disc SCI Loss: 1.347268
Step 257: Disc SCI Loss: 1.339511
Step 258: Disc SCI Loss: 1.340100
Step 259: Disc SCI Loss: 1.345749
Step 260: Disc SCI Loss: 1.340135
Step 261: Disc SCI Loss: 1.339192
Step 262: Disc SCI Loss: 1.342158
Step 263: Disc SCI Loss: 1.344336
Step 264: Disc SCI Loss: 1.337589
Step 265: Disc SCI Loss: 1.335170
Step 266: Disc SCI Loss: 1.342720
Step 267: Disc

Step 474: Disc SCI Loss: 1.284802
Step 475: Disc SCI Loss: 1.292468
Step 476: Disc SCI Loss: 1.296695
Step 477: Disc SCI Loss: 1.290930
Step 478: Disc SCI Loss: 1.298515
Step 479: Disc SCI Loss: 1.287073
Step 480: Disc SCI Loss: 1.290472
Step 481: Disc SCI Loss: 1.291989
Step 482: Disc SCI Loss: 1.284108
Step 483: Disc SCI Loss: 1.281921
Step 484: Disc SCI Loss: 1.301304
Step 485: Disc SCI Loss: 1.293208
Step 486: Disc SCI Loss: 1.279709
Step 487: Disc SCI Loss: 1.277967
Step 488: Disc SCI Loss: 1.284688
Step 489: Disc SCI Loss: 1.283300
Step 490: Disc SCI Loss: 1.276428
Step 491: Disc SCI Loss: 1.283571
Step 492: Disc SCI Loss: 1.274863
Step 493: Disc SCI Loss: 1.281653
Step 494: Disc SCI Loss: 1.291013
Step 495: Disc SCI Loss: 1.274953
Step 496: Disc SCI Loss: 1.285677
Step 497: Disc SCI Loss: 1.279795
Step 498: Disc SCI Loss: 1.295417
Step 499: Disc SCI Loss: 1.292313
Step 500: Disc SCI Loss: 1.288604
Model saved in path: ../../models/GAN_scionly_1.29loss_500steps_2019_03_11_20_56_0

Step 700: Disc SCI Loss: 1.244997
Model saved in path: ../../models/GAN_scionly_1.24loss_700steps_2019_03_11_20_56_47.ckpt
Step 701: Disc SCI Loss: 1.264520
Step 702: Disc SCI Loss: 1.230390
Step 703: Disc SCI Loss: 1.225779
Step 704: Disc SCI Loss: 1.216242
Step 705: Disc SCI Loss: 1.228227
Step 706: Disc SCI Loss: 1.247634
Step 707: Disc SCI Loss: 1.241911
Step 708: Disc SCI Loss: 1.222595
Step 709: Disc SCI Loss: 1.261239
Step 710: Disc SCI Loss: 1.230720
Step 711: Disc SCI Loss: 1.219783
Step 712: Disc SCI Loss: 1.240310
Step 713: Disc SCI Loss: 1.240564
Step 714: Disc SCI Loss: 1.240708
Step 715: Disc SCI Loss: 1.234600
Step 716: Disc SCI Loss: 1.218970
Step 717: Disc SCI Loss: 1.231693
Step 718: Disc SCI Loss: 1.225402
Step 719: Disc SCI Loss: 1.229448
Step 720: Disc SCI Loss: 1.243743
Step 721: Disc SCI Loss: 1.220880
Step 722: Disc SCI Loss: 1.243498
Step 723: Disc SCI Loss: 1.240041
Step 724: Disc SCI Loss: 1.231103
Step 725: Disc SCI Loss: 1.224972
Step 726: Disc SCI Loss: 1.

Step 934: Disc SCI Loss: 1.199704
Step 935: Disc SCI Loss: 1.194788
Step 936: Disc SCI Loss: 1.195657
Step 937: Disc SCI Loss: 1.184305
Step 938: Disc SCI Loss: 1.191056
Step 939: Disc SCI Loss: 1.196043
Step 940: Disc SCI Loss: 1.194053
Step 941: Disc SCI Loss: 1.177930
Step 942: Disc SCI Loss: 1.191179
Step 943: Disc SCI Loss: 1.201114
Step 944: Disc SCI Loss: 1.194365
Step 945: Disc SCI Loss: 1.195508
Step 946: Disc SCI Loss: 1.196746
Step 947: Disc SCI Loss: 1.203453
Step 948: Disc SCI Loss: 1.181180
Step 949: Disc SCI Loss: 1.191393
Step 950: Disc SCI Loss: 1.196545
Step 951: Disc SCI Loss: 1.179058
Step 952: Disc SCI Loss: 1.208000
Step 953: Disc SCI Loss: 1.201657
Step 954: Disc SCI Loss: 1.174483
Step 955: Disc SCI Loss: 1.174794
Step 956: Disc SCI Loss: 1.203877
Step 957: Disc SCI Loss: 1.194280
Step 958: Disc SCI Loss: 1.205260
Step 959: Disc SCI Loss: 1.176831
Step 960: Disc SCI Loss: 1.193597
Step 961: Disc SCI Loss: 1.199918
Step 962: Disc SCI Loss: 1.196474
Step 963: Disc

...

Step 27579: Generator Loss: 0.696467, Disc gvr Loss: 1.386338
Step 27580: Generator Loss: 0.696467, Disc gvr Loss: 1.386325
Step 27581: Generator Loss: 0.696467, Disc gvr Loss: 1.386372
Step 27582: Generator Loss: 0.696466, Disc gvr Loss: 1.386345
Step 27583: Generator Loss: 0.696466, Disc gvr Loss: 1.386345
Step 27584: Generator Loss: 0.696466, Disc gvr Loss: 1.386390
Step 27585: Generator Loss: 0.696465, Disc gvr Loss: 1.386310
Step 27586: Generator Loss: 0.696465, Disc gvr Loss: 1.386340
Step 27587: Generator Loss: 0.696465, Disc gvr Loss: 1.386330
Step 27588: Generator Loss: 0.696464, Disc gvr Loss: 1.386345
Step 27589: Generator Loss: 0.696464, Disc gvr Loss: 1.386336
Step 27590: Generator Loss: 0.696464, Disc gvr Loss: 1.386320
Step 27591: Generator Loss: 0.696463, Disc gvr Loss: 1.386331
Step 27592: Generator Loss: 0.696463, Disc gvr Loss: 1.386326
Step 27593: Generator Loss: 0.696463, Disc gvr Loss: 1.386366
Step 27594: Generator Loss: 0.696462, Disc gvr Loss: 1.386307
Step 275

Step 27709: Generator Loss: 0.696424, Disc gvr Loss: 1.386323
Step 27710: Generator Loss: 0.696424, Disc gvr Loss: 1.386356
Step 27711: Generator Loss: 0.696424, Disc gvr Loss: 1.386311
Step 27712: Generator Loss: 0.696423, Disc gvr Loss: 1.386337
Step 27713: Generator Loss: 0.696423, Disc gvr Loss: 1.386334
Step 27714: Generator Loss: 0.696423, Disc gvr Loss: 1.386357
Step 27715: Generator Loss: 0.696422, Disc gvr Loss: 1.386332
Step 27716: Generator Loss: 0.696422, Disc gvr Loss: 1.386357
Step 27717: Generator Loss: 0.696422, Disc gvr Loss: 1.386328
Step 27718: Generator Loss: 0.696421, Disc gvr Loss: 1.386327
Step 27719: Generator Loss: 0.696421, Disc gvr Loss: 1.386362
Step 27720: Generator Loss: 0.696421, Disc gvr Loss: 1.386333
Step 27721: Generator Loss: 0.696420, Disc gvr Loss: 1.386349
Step 27722: Generator Loss: 0.696420, Disc gvr Loss: 1.386003
Step 27723: Generator Loss: 0.696420, Disc gvr Loss: 1.386337
Step 27724: Generator Loss: 0.696419, Disc gvr Loss: 1.386323
Step 277

Step 27840: Generator Loss: 0.696381, Disc gvr Loss: 1.386314
Step 27841: Generator Loss: 0.696381, Disc gvr Loss: 1.386371
Step 27842: Generator Loss: 0.696380, Disc gvr Loss: 1.386320
Step 27843: Generator Loss: 0.696380, Disc gvr Loss: 1.386354
Step 27844: Generator Loss: 0.696380, Disc gvr Loss: 1.386320
Step 27845: Generator Loss: 0.696379, Disc gvr Loss: 1.386341
Step 27846: Generator Loss: 0.696379, Disc gvr Loss: 1.386318
Step 27847: Generator Loss: 0.696379, Disc gvr Loss: 1.386353
Step 27848: Generator Loss: 0.696378, Disc gvr Loss: 1.386373
Step 27849: Generator Loss: 0.696378, Disc gvr Loss: 1.386328
Step 27850: Generator Loss: 0.696378, Disc gvr Loss: 1.386317
Step 27851: Generator Loss: 0.696377, Disc gvr Loss: 1.386342
Step 27852: Generator Loss: 0.696377, Disc gvr Loss: 1.386368
Step 27853: Generator Loss: 0.696377, Disc gvr Loss: 1.386343
Step 27854: Generator Loss: 0.696376, Disc gvr Loss: 1.386360
Step 27855: Generator Loss: 0.696376, Disc gvr Loss: 1.386315
Step 278

Step 27971: Generator Loss: 0.696338, Disc gvr Loss: 1.386351
Step 27972: Generator Loss: 0.696338, Disc gvr Loss: 1.386318
Step 27973: Generator Loss: 0.696338, Disc gvr Loss: 1.386332
Step 27974: Generator Loss: 0.696337, Disc gvr Loss: 1.386386
Step 27975: Generator Loss: 0.696337, Disc gvr Loss: 1.386340
Step 27976: Generator Loss: 0.696337, Disc gvr Loss: 1.386323
Step 27977: Generator Loss: 0.696336, Disc gvr Loss: 1.386320
Step 27978: Generator Loss: 0.696336, Disc gvr Loss: 1.386369
Step 27979: Generator Loss: 0.696336, Disc gvr Loss: 1.386387
Step 27980: Generator Loss: 0.696335, Disc gvr Loss: 1.386373
Step 27981: Generator Loss: 0.696335, Disc gvr Loss: 1.386325
Step 27982: Generator Loss: 0.696335, Disc gvr Loss: 1.386339
Step 27983: Generator Loss: 0.696334, Disc gvr Loss: 1.386331
Step 27984: Generator Loss: 0.696334, Disc gvr Loss: 1.386324
Step 27985: Generator Loss: 0.696334, Disc gvr Loss: 1.386358
Step 27986: Generator Loss: 0.696333, Disc gvr Loss: 1.386314
Step 279

Step 28101: Generator Loss: 0.696297, Disc gvr Loss: 1.386386
Step 28102: Generator Loss: 0.696296, Disc gvr Loss: 1.386233
Step 28103: Generator Loss: 0.696296, Disc gvr Loss: 1.386328
Step 28104: Generator Loss: 0.696296, Disc gvr Loss: 1.386366
Step 28105: Generator Loss: 0.696295, Disc gvr Loss: 1.386349
Step 28106: Generator Loss: 0.696295, Disc gvr Loss: 1.386369
Step 28107: Generator Loss: 0.696295, Disc gvr Loss: 1.386317
Step 28108: Generator Loss: 0.696294, Disc gvr Loss: 1.386315
Step 28109: Generator Loss: 0.696294, Disc gvr Loss: 1.386323
Step 28110: Generator Loss: 0.696294, Disc gvr Loss: 1.386331
Step 28111: Generator Loss: 0.696293, Disc gvr Loss: 1.386325
Step 28112: Generator Loss: 0.696293, Disc gvr Loss: 1.383338
Step 28113: Generator Loss: 0.696293, Disc gvr Loss: 1.386405
Step 28114: Generator Loss: 0.696292, Disc gvr Loss: 1.385835
Step 28115: Generator Loss: 0.696292, Disc gvr Loss: 1.386391
Step 28116: Generator Loss: 0.696292, Disc gvr Loss: 1.386315
Step 281

So we are getting a diverging GAN error, according to https://github.com/soumith/ganhacks/issues/14, this may be because

Pretraining the generator with real susp conversations is not working since in training, noise is used as generator's input

If discriminator's loss keeps increasing, it is because noise added to inputs into discriminator is too much.

Things done:
1. Change loss optimization eqn of Disc to SGD instead of Adam
2. Train generator twice for every time disc is trained
3. Add noise to discriminator

In [ ]:
    print('Pretraining of generator and disc_gvr starting.')
    for i in range(1, num_steps*100+1):
        z = np.random.uniform(-1., 1., size=[batch_size, noise_dim])
        feat_noise = np.random.uniform(-0.01, 0.01, size=[batch_size, noise_dim])
        feed_dict = {gen_input: z, features_noise: feat_noise}
        _, _, gl, dgvrl = sess.run([train_gen, train_disc_gvr, gen_loss, disc_gvr_loss], feed_dict=feed_dict)
        _, gl = sess.run([train_gen, gen_loss], feed_dict=feed_dict)
        print('Step %i: Generator Loss: %f, Disc gvr Loss: %f' % (i, gl, dgvrl))
        if i % 100 == 0:
            # Save the variables to disk.
            filename = '../../models/GAN_gengvr_' + "{:.2f}loss_{}steps_".format(gl, i) + datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")  + '.ckpt'
            save_path = saver.save(sess, filename)
            print("Model saved in path: %s" % save_path)
    